In [ ]:
import bioread
import neurokit2 as nk
import pandas as pd
import numpy as np
import pyhrv
import matplotlib
matplotlib.use('Agg')

In [ ]:
participants_selected = ['pp2', 'pp3', 'pp4', 'pp6', 'pp22', 'pp30', 'pp31', 'pp24', 'pp25', 'pp45', 'pp53', 'pp47', 'pp49', 'pp56', 'pp66', 'pp75', 'pp70', 'pp85', 'pp93', 'pp102', 'pp96', 'pp104', 'pp103', 'pp91', 'pp115', 'pp107', 'pp110', 'pp112', "pp95", "pp97"]


In [ ]:

duration_matrix = pd.read_csv(r"C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\durationtable.csv")

results1 = []
results2 = []
results3 = []

In [ ]:
results1 = []

for participant in participants_selected:
    acq_file_path = r"C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\wavescontrolled\waves\LOG"+participant+".acq"
    print(participant)
    intervals = []
    start_time = 0
    durations = duration_matrix[participant].tolist()
    for duration in durations:
        end_time = start_time + duration
        intervals.append((start_time, end_time))
        start_time = end_time

    # Read ECG data
    ecg_channel_name = "final ECG"
    file = bioread.read_file(acq_file_path)
    channel = next(c for c in file.channels if ecg_channel_name in c.name)
    ecg_data = channel.data
    timestamps = channel.time_index
    sampling_rate = channel.samples_per_second

    videonr = 0
    baseline_hr = None
    prev_metrics = {}

    interval_metrics = []

    for start, end in intervals:
        idx = np.where((timestamps >= start) & (timestamps < end))[0]

        if len(idx) < int(sampling_rate * 5):
            continue

        segment = ecg_data[idx]
        try:
            signals, info = nk.ecg_process(segment, sampling_rate=sampling_rate)
            heart_rate_series = signals["ECG_Rate"]
            mean_hr = heart_rate_series.mean()
            rpeaks = info["ECG_R_Peaks"]

            if len(rpeaks) < 2:
                continue

            time_features = nk.hrv_time(rpeaks, sampling_rate=sampling_rate, show=False)

            # Baseline
            if baseline_hr is None:
                baseline_hr = mean_hr
                baseline_RMSSD = time_features.get("HRV_RMSSD", [np.nan])[0]
                baseline_SDSD = time_features.get("HRV_SDSD", [np.nan])[0]
                baseline_MeanNN = time_features.get("HRV_MeanNN", [np.nan])[0]
                baseline_MedianNN = time_features.get("HRV_MedianNN", [np.nan])[0]
                baseline_SDNN = time_features.get("HRV_SDNN", [np.nan])[0]
                baseline_SDRMSSD = time_features.get("HRV_SDRMSSD", [np.nan])[0]

            try:
                welch_results = pyhrv.frequency_domain.welch_psd(rpeaks, show=False)
                lf_hf_ratio_pyhrv = float(welch_results["fft_ratio"])
            except Exception as e:
                lf_hf_ratio_pyhrv = np.nan

            curr_metrics = {
                "RMSSD": time_features.get("HRV_RMSSD", [np.nan])[0],
                "SDSD": time_features.get("HRV_SDSD", [np.nan])[0],
                "MeanHR": mean_hr,
                "MeanNN": time_features.get("HRV_MeanNN", [np.nan])[0],
                "MedianNN": time_features.get("HRV_MedianNN", [np.nan])[0],
                "SDNN": time_features.get("HRV_SDNN", [np.nan])[0],
                "SDRMSSD": time_features.get("HRV_SDRMSSD", [np.nan])[0],
                "LF/HF": lf_hf_ratio_pyhrv,
                "CVNN": time_features.get("HRV_CVNN", [np.nan])[0],
                "CVSD": time_features.get("HRV_CVSD", [np.nan])[0],
            }

            # Baseline diff
            curr_metrics["RMSSD_diff"] = curr_metrics["RMSSD"] - baseline_RMSSD
            curr_metrics["SDSD_diff"] = curr_metrics["SDSD"] - baseline_SDSD
            curr_metrics["MeanHR_diff"] = curr_metrics["MeanHR"] - baseline_hr
            curr_metrics["MeanNN_diff"] = curr_metrics["MeanNN"] - baseline_MeanNN
            curr_metrics["MedianNN_diff"] = curr_metrics["MedianNN"] - baseline_MedianNN
            curr_metrics["SDNN_diff"] = curr_metrics["SDNN"] - baseline_SDNN
            curr_metrics["SDRMSSD_diff"] = curr_metrics["SDRMSSD"] - baseline_SDRMSSD

            base_metric_keys = ["RMSSD", "SDSD", "MeanHR", "MeanNN", "MedianNN", "SDNN", "SDRMSSD", "LF/HF", "CVNN", "CVSD"]

            for k in base_metric_keys:
                if k in prev_metrics and isinstance(curr_metrics[k], (int, float)) and isinstance(prev_metrics[k], (int, float)):
                    curr_metrics[f"{k}_prev_diff"] = curr_metrics[k] - prev_metrics[k]
                else:
                    curr_metrics[f"{k}_prev_diff"] = 0


            curr_metrics.update({
                "participant": participant,
                "videonr": videonr,
                "Start Time (s)": start,
                "End Time (s)": end
            })

            interval_metrics.append(curr_metrics)
            prev_metrics = curr_metrics.copy()
            print(videonr)
            videonr += 1

        except Exception as e:
            print(f"Failed interval ({start}-{end}): {e}")
            continue
    df = pd.DataFrame(interval_metrics)
    feature_cols = ["RMSSD", "SDSD", "MeanHR", "MeanNN", "MedianNN", "SDNN", "SDRMSSD", "LF/HF", "CVNN", "CVSD"]

    means = df[feature_cols].mean()
    medians = df[feature_cols].median()

    for m in interval_metrics:
        for f in feature_cols:
            m[f"{f}_mean_diff"] = m[f] - means[f]
            m[f"{f}_median_diff"] = m[f] - medians[f]
    print(means, medians)
    results1.extend(interval_metrics)


In [ ]:
results2 = []

for participant in participants_selected:
    acq_file_path = r"C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\wavescontrolled\waves\LOG"+participant+".acq"

    intervals = []
    start_time = 0
    durations = duration_matrix[participant].tolist()

    group_size = 3
    for i in range(0, len(durations), group_size):
        group = durations[i:i + group_size]
        duration_sum = sum(group)
        end_time = start_time + duration_sum
        intervals.append((start_time, end_time))
        start_time = end_time

    # Read ECG data
    ecg_channel_name = "final ECG"
    file = bioread.read_file(acq_file_path)
    channel = next(c for c in file.channels if ecg_channel_name in c.name)
    ecg_data = channel.data
    timestamps = channel.time_index
    sampling_rate = channel.samples_per_second

    videonr = 0
    baseline_hr = None
    prev_metrics = {}

    interval_metrics = []

    for start, end in intervals:
        idx = np.where((timestamps >= start) & (timestamps < end))[0]

        if len(idx) < int(sampling_rate * 5):
            continue

        segment = ecg_data[idx]
        try:
            signals, info = nk.ecg_process(segment, sampling_rate=sampling_rate)
            heart_rate_series = signals["ECG_Rate"]
            mean_hr = heart_rate_series.mean()
            rpeaks = info["ECG_R_Peaks"]

            if len(rpeaks) < 2:
                continue

            time_features = nk.hrv_time(rpeaks, sampling_rate=sampling_rate, show=False)

            # Baseline
            if baseline_hr is None:
                baseline_hr = mean_hr
                baseline_RMSSD = time_features.get("HRV_RMSSD", [np.nan])[0]
                baseline_SDSD = time_features.get("HRV_SDSD", [np.nan])[0]
                baseline_MeanNN = time_features.get("HRV_MeanNN", [np.nan])[0]
                baseline_MedianNN = time_features.get("HRV_MedianNN", [np.nan])[0]
                baseline_SDNN = time_features.get("HRV_SDNN", [np.nan])[0]
                baseline_SDRMSSD = time_features.get("HRV_SDRMSSD", [np.nan])[0]

            try:
                welch_results = pyhrv.frequency_domain.welch_psd(rpeaks, show=False)
                lf_hf_ratio_pyhrv = float(welch_results["fft_ratio"])
            except Exception as e:
                lf_hf_ratio_pyhrv = np.nan

            # Add _3 suffix to all metric keys:
            curr_metrics = {
                "RMSSD_3": time_features.get("HRV_RMSSD", [np.nan])[0],
                "SDSD_3": time_features.get("HRV_SDSD", [np.nan])[0],
                "MeanHR_3": mean_hr,
                "MeanNN_3": time_features.get("HRV_MeanNN", [np.nan])[0],
                "MedianNN_3": time_features.get("HRV_MedianNN", [np.nan])[0],
                "SDNN_3": time_features.get("HRV_SDNN", [np.nan])[0],
                "SDRMSSD_3": time_features.get("HRV_SDRMSSD", [np.nan])[0],
                "LF/HF_3": lf_hf_ratio_pyhrv,
                "CVNN_3": time_features.get("HRV_CVNN", [np.nan])[0],
                "CVSD_3": time_features.get("HRV_CVSD", [np.nan])[0],
            }
            curr_metrics["RMSSD_diff_3"] = curr_metrics["RMSSD_3"] - baseline_RMSSD
            curr_metrics["SDSD_diff_3"] = curr_metrics["SDSD_3"] - baseline_SDSD
            curr_metrics["MeanHR_diff_3"] = curr_metrics["MeanHR_3"] - baseline_hr
            curr_metrics["MeanNN_diff_3"] = curr_metrics["MeanNN_3"] - baseline_MeanNN
            curr_metrics["MedianNN_diff_3"] = curr_metrics["MedianNN_3"] - baseline_MedianNN
            curr_metrics["SDNN_diff_3"] = curr_metrics["SDNN_3"] - baseline_SDNN
            curr_metrics["SDRMSSD_diff_3"] = curr_metrics["SDRMSSD_3"] - baseline_SDRMSSD
            base_metric_keys = ["RMSSD_3", "SDSD_3", "MeanHR_3", "MeanNN_3", "MedianNN_3", "SDNN_3", "SDRMSSD_3", "LF/HF_3", "CVNN_3", "CVSD_3"]

            for k in base_metric_keys:
                if k in prev_metrics and isinstance(curr_metrics[k], (int, float)) and isinstance(prev_metrics[k], (int, float)):
                    curr_metrics[f"{k}_prev_diff_3"] = curr_metrics[k] - prev_metrics[k]
                else:
                    curr_metrics[f"{k}_prev_diff_3"] = 0
            for x in range(group_size):
                print(videonr)
                curr_metrics.update({
                    "participant": participant,
                    "videonr": videonr,
                    "Start Time (s)": start,
                    "End Time (s)": end
                })
                interval_metrics.append(curr_metrics.copy())
                videonr += 1
            prev_metrics = curr_metrics.copy()

        except Exception as e:
            print(f"Failed interval ({start}-{end}): {e}")
            continue
    df = pd.DataFrame(interval_metrics)
    feature_cols = ["RMSSD_3", "SDSD_3", "MeanHR_3", "MeanNN_3", "MedianNN_3", "SDNN_3", "SDRMSSD_3", "LF/HF_3", "CVNN_3", "CVSD_3"]

    means = df[feature_cols].mean()
    medians = df[feature_cols].median()

    for m in interval_metrics:
        for f in feature_cols:
            m[f"{f}_mean_diff_3"] = m[f] - means[f]
            m[f"{f}_median_diff_3"] = m[f] - medians[f]
    results2.extend(interval_metrics)


In [ ]:
print(results2[:10])

In [ ]:
print(results2[:10])

In [ ]:
'''
for participant in participants_selected:
    intervals = []
    durations = duration_matrix[participant].tolist()
    start_time =0
    group_size=6
    for i in range(0, len(durations), group_size):
        group = durations[i:i + group_size]
        duration_sum = sum(group)
        end_time = start_time + duration_sum
        intervals.append((start_time, end_time))
        start_time = end_time
    print(intervals)
    acq_file_path = r"C:/Users/20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\wavescontrolled\waves\LOG"+participant+".acq"
    ecg_channel_name = "final ECG"
    file = bioread.read_file(acq_file_path)
    channel = next(c for c in file.channels if ecg_channel_name in c.name)
    ecg_data = channel.data
    timestamps = channel.time_index
    sampling_rate = channel.samples_per_second
    videonr = 0

    for start, end in intervals:
        idx = np.where((timestamps >= start) & (timestamps < end))[0]

        if len(idx) < int(sampling_rate*5):
            print(f"Skipping interval ({start}, {end}) — not enough data.")
            continue

        segment = ecg_data[idx]

        try:
            signals, info = nk.ecg_process(segment, sampling_rate=sampling_rate)
            rpeaks = info["ECG_R_Peaks"]

            if len(rpeaks) < 2:
                print(f"Skipping interval ({start}-{end}) — not enough R-peaks.")
                continue

            # Time-domain metrics
            print(f"Interval ({start}-{end}): {len(rpeaks)} R-peaks detected")
            try:
                time_features = nk.hrv_time(rpeaks, sampling_rate=sampling_rate, show=False)
            except Exception as e:
                print(f"Time HRV failed: {e}")
                time_features = pd.DataFrame()
            # Frequency-domain metrics
            try:
                welch_results = pyhrv.frequency_domain.welch_psd(rpeaks, show=False)
                lf_hf_ratio_pyhrv = float(welch_results["fft_ratio"])
            except Exception as e:
                print(f"Frequency HRV failed: {e}")
                freq_features = pd.DataFrame()

            # Use .get() with default values
            for x in range(group_size):
                results3.append({
                    "participant" : participant,
                    "videonr" : videonr,
                    "Start Time (s) 60s": start,
                    "End Time (s) 60s": end,
                    "RMSSD_6": time_features.get("HRV_RMSSD", [np.nan])[0],
                    "SDSD_6": time_features.get("HRV_SDSD", [np.nan])[0],
                    "LF/HF_6": lf_hf_ratio_pyhrv
                })
                videonr = videonr + 1


        except Exception as e:
            print(f"Failed to process interval ({start}-{end}): {e}")
            continue
'''

In [ ]:
'''
results1_df = pd.DataFrame(results1)
results2_df = pd.DataFrame(results2)
results3_df = pd.DataFrame(results3)
# Merge results1_df and results2_df
merged_df = pd.merge(
    results1_df,
    results2_df,
    on=['participant', 'videonr'],
    how='inner'
)

# Merge the result with results3_df
df_results = pd.merge(
    merged_df,
    results3_df,
    on=['participant', 'videonr'],
    how='inner'
)
df_results.to_csv("hrv_all.csv", index=False)
results1_df.to_csv("hrv_1.csv", index=False)
results2_df.to_csv("hrv_3.csv", index=False)
'''


In [ ]:
results1_df = pd.DataFrame(results1)
results2_df = pd.DataFrame(results2)
# Merge results1_df and results2_df
merged_df = pd.merge(
    results1_df,
    results2_df,
    on=['participant', 'videonr'],
    how='inner'
)
merged_df.to_csv("hrv_all.csv", index=False)
results1_df.to_csv("hrv_1.csv", index=False)
results2_df.to_csv("hrv_3.csv", index=False)